In [1]:
!pip install accelerate bitsandbytes transformers peft datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0


In [3]:
import pandas as pd
df = pd.read_csv('parsing.csv')
documents = df.apply(lambda row: f"Q: {row['Question']}\nA: {row['Answer']}", axis=1).tolist()


In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2", cache_folder="downloader")
embeddings = model.encode(documents, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.2 MB/s eta 0:00:00


In [9]:
import faiss
import numpy as np
import pickle

# Ensure 'embeddings' is available from previous steps (e.g., cell V1-D7iT4sqop)
# For demonstration purposes, assuming 'embeddings' is a torch tensor. Convert to numpy if needed.
if hasattr(embeddings, 'cpu'):
    embeddings_np = embeddings.cpu().numpy()
else:
    embeddings_np = np.array(embeddings)

# Build the FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

# Save the FAISS index to a pkl file using pickle
with open("index.pkl", "wb") as f:
    pickle.dump(index, f)

print("FAISS index built and saved to index.pkl")

# Save FAISS index in its native format
faiss.write_index(index, "index.faiss")

print("FAISS index saved to index.faiss")

FAISS index built and saved to index.pkl
FAISS index saved to index.faiss


In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
model_id = "Norquinal/Mistral-7B-claude-chat"
tokenizer = AutoTokenizer.from_pretrained("Norquinal/Mistral-7B-claude-chat", cache_dir="downloader")
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16, offload_folder="/offload", cache_dir="downloader")

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [25]:
def generate_answer(query, index, documents, embed_model, lm_model, tokenizer, max_tokens=300):
    import numpy as np
    import time

    # Step 1: Embed the query
    query_embedding = embed_model.encode(query)

    # Step 2: Retrieve top-k chunks
    D, I = index.search(np.array([query_embedding]), k=5)

    # Step 3: Find the most relevant chunk that contains the exact question
    matching_chunk = None
    for i in I[0]:
        doc = documents[i]
        if query.lower() in doc.lower():
            matching_chunk = doc
            break

    # Step 4: If exact QA pair is found, return the answer directly
    if matching_chunk:
        print("✅ Direct match found. Returning extracted answer.\n")
        for line in matching_chunk.split("\n"):
            if line.strip().lower().startswith("a:"):
                return line.replace("A:", "Answer:").strip()
        return matching_chunk  # fallback if A: line isn’t isolated

    # Step 5: Otherwise, continue with generation using retrieved chunks
    def strip_qa_noise(text):
        lines = text.split("\n")
        if len(lines) <= 3:
            return text
        return "\n".join(line for line in lines if not line.strip().startswith(("Q:", "A:", "Question:", "Answer:")))

    cleaned_chunks = [strip_qa_noise(documents[i]) for i in I[0]]
    context = "\n".join(cleaned_chunks)

    prompt = f"""You are a helpful technical assistant.

Use only the following context to answer the current question. Do not repeat unrelated prior questions.

Context:
{context}

Question: {query}
Answer:"""

    device = next(lm_model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = lm_model.generate(**inputs, max_new_tokens=max_tokens)

    return tokenizer.decode(output[0], skip_special_tokens=True)




generate_answer(
    query="What should be done if the edge/cluster fails to come up after a management interface toggle (ONECELL5G-1068)?",
    index=index,
    documents=documents,
    embed_model=SentenceTransformer("all-MiniLM-L6-v2"),
    lm_model=model,
    tokenizer=tokenizer
)



✅ Direct match found. Returning extracted answer.



'Answer: 1. Power off the Standby node. 2. Reboot the Active node. 3. Once the Active node is up, power on the Standby node.'

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
